In [7]:
import numpy as np
import onnxruntime as ort
import cv2
import mlflow
from mlflow.tracking import MlflowClient

In [8]:
MLFLOW_URI = "http://localhost:8080"
MODEL_NAME = "anatomy_detector"
ALIAS = "champion"
TARGET_HEIGHT = 400
TARGET_WIDTH = 600

In [9]:
mlflow.set_tracking_uri(MLFLOW_URI)
client = MlflowClient()
model_version = client.get_model_version_by_alias(name=MODEL_NAME, alias=ALIAS)
onnx_source = model_version.source
ONNX_MODEL_PATH = mlflow.artifacts.download_artifacts(onnx_source)

In [6]:
# Load ONNX model
ort_session = ort.InferenceSession(ONNX_MODEL_PATH)

# test with image 1528.png 
test_img_path = r"dataset/Dataset for Fetus Framework/Internal Test Set/Standard/1528.png"

# Preprocess the image
test_img = cv2.imread(test_img_path, cv2.IMREAD_GRAYSCALE)      # Load as grayscale
test_img = cv2.resize(test_img, (TARGET_WIDTH, TARGET_HEIGHT))  # Resize to target dimensions
test_img = ((test_img / 255.0 - 0.5) / 0.5).astype(np.float32)                # Normalize to [0, 1]
test_img = np.expand_dims(test_img, axis=0)                     # Add batch dimension
test_img = np.expand_dims(test_img, axis=0)                     # Add channel dimension

# Run inference
ort_inputs = {'input': test_img}
ort_outputs = ort_session.run(None, ort_inputs)
pred_class_conf = np.array(ort_outputs[0].squeeze(0))   # [K]
pred_boxes = np.array(ort_outputs[1].squeeze(0))        # [K, 4]

# Print results
print(f"Predicted class confidences: {pred_class_conf}")
print(f"Predicted boxes: {pred_boxes}")

Predicted class confidences: [0.8649031  0.93156165 0.9503524  0.96955484 0.9971384  0.99441046
 0.9998605  0.9509804  0.9894701 ]
Predicted boxes: [[0.37585613 0.7974008  0.14016238 0.05754925]
 [0.35195756 0.736306   0.10226785 0.06196683]
 [0.5787262  0.8576854  0.519773   0.07402731]
 [0.27359235 0.7032237  0.10246657 0.16758417]
 [0.5011482  0.3334785  0.06206264 0.01847832]
 [0.5168424  0.30724606 0.04395769 0.01814534]
 [0.5407074  0.30092418 0.04477595 0.01912759]
 [0.5282779  0.4572421  0.15927117 0.13536735]
 [0.30684185 0.51673836 0.12966292 0.14619088]]
